In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist

#Load dataset as train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Using TensorFlow backend.


11493376/11490434 [==============================] - 1s 0us/step


In [2]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
num_classes = 10
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [3]:
from keras.models import Sequential
from keras import models, layers
from keras import regularizers
model = Sequential()
model.add(layers.Dropout(0.2,input_shape=(784,)))
model.add(layers.Dense(1000,
                        kernel_regularizer = regularizers.l2(0.01),
                        activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1000,
                        kernel_regularizer = regularizers.l2(0.01),
                        activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10,  activation='softmax'))
#display the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_1 (Dropout)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                10010     
Total params: 1,796,010
Trainable params: 1,796,010
Non-trainable params: 0
____________________________________________

In [4]:
model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer='adam', 
              metrics=['accuracy'])

In [5]:
hist = model.fit(x_train, y_train,
                        batch_size=128,
                        epochs=100,
                        verbose=1,
                        validation_data=(x_test,y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 3s 51us/step - loss: 2.0675 - accuracy: 0.8560 - val_loss: 0.6820 - val_accuracy: 0.9177
Epoch 2/100
60000/60000 [==============================] - 2s 32us/step - loss: 0.7667 - accuracy: 0.8899 - val_loss: 0.6296 - val_accuracy: 0.9351
Epoch 3/100
60000/60000 [==============================] - 2s 32us/step - loss: 0.7300 - accuracy: 0.8975 - val_loss: 0.6505 - val_accuracy: 0.9228
Epoch 4/100
60000/60000 [==============================] - 2s 33us/step - loss: 0.7061 - accuracy: 0.9017 - val_loss: 0.5608 - val_accuracy: 0.9432
Epoch 5/100
60000/60000 [==============================] - 2s 33us/step - loss: 0.6862 - accuracy: 0.9056 - val_loss: 0.5648 - val_accuracy: 0.9418
Epoch 6/100
60000/60000 [==============================] - 2s 32us/step - loss: 0.6779 - accuracy: 0.9062 - val_loss: 0.5522 - val_accuracy: 0.9437
Epoch 7/100
60000/60000 [==============================] - 2s 

In [7]:
score = model.evaluate(x_test, y_test, verbose=1)
print("Test loss {:.4f}, accuracy {:.2f}%".format(score[0], score[1] * 100))

10000/10000 [==============================] - 1s 56us/step
Test loss 0.3956, accuracy 95.60%


In [8]:
#Save the entire model in model.h5 file
model.save("model.h5")
print("Saved model to disk")

Saved model to disk


In [9]:
model = tf.keras.models.load_model('model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
#saving converted model in "converted_model.tflite" file
open("converted_model.tflite", "wb").write(tflite_model)

7185660

In [10]:
model = tf.keras.models.load_model('model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
#saving converted model in "converted_quant_model.tflite" file
open("converted_quant_model.tflite", "wb").write(tflite_quant_model)

1803768

In [17]:
# Load TFLite model and allocate tensors.
interpreter = \
tf.lite.Interpreter(model_path="converted_quant_model.tflite")
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Test model on some input data.
input_shape = input_details[0]['shape']
acc=0
for i in range(len(x_test)):
    input_data = x_test[i].reshape(input_shape)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    if(np.argmax(output_data) == np.argmax(y_test[i])):
        acc+=1
acc = acc/len(x_test)
print(acc*100)

95.6


# VGG on CIFAR10

In [18]:
from keras.datasets import cifar10
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [20]:
from keras.applications import vgg16
IMG_SHAPE = x_train[0].shape
VGG16_MODEL=tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
VGG16_MODEL.trainable=False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(10,activation='softmax')

58892288/58889256 [==============================] - 2s 0us/step


In [21]:
model = tf.keras.Sequential([
  VGG16_MODEL,
  global_average_layer,
  prediction_layer
])

In [22]:
model.compile(optimizer='adam', 
              loss=keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

In [23]:
hist = model.fit(x_train, y_train,
                        batch_size=128,
                        epochs=100,
                        verbose=0,
                        validation_data=(x_test,y_test))

In [24]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss {:.4f}, accuracy {:.2f}%".format(score[0], score[1] * 100))

Test loss 1.4478, accuracy 56.47%


In [25]:
model.save("vgg_model.h5")
print("Saved model to disk")

Saved model to disk


In [26]:
model = tf.keras.models.load_model('vgg_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
open("vgg_quant_model.tflite", "wb").write(tflite_quant_model)

14740040

In [29]:
x_test = x_test.astype('float32')

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="vgg_quant_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on some input data.
input_shape = input_details[0]['shape']
acc=0
for i in range(len(x_test)):
    input_data = x_test[i].reshape(input_shape)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    if(np.argmax(output_data) == y_test[i]):
        acc+=1
acc = acc/len(x_test)
print(acc*100)

41.75
